In [1]:
import pandas as pd

In [2]:
# Read in data
df = pd.read_csv("../csv_output/mechanism_nlp.csv")
df.head()

,patent_number,patent_title
0,RE30525,Extended range hydraulic transmission
1,RE30135,Electric fail-safe actuator
2,RE29872,Differential gear mechanism
3,RE30334,Pressure compensated hermetically sealed trans...
4,RE30120,Lobe type pump adjustment


In [3]:
num, title=next(df.itertuples(index=False))
num, title

('RE30525', 'Extended range hydraulic transmission')

In [4]:
patent_text=[]

In [5]:
for patent_number, patent_title in df.itertuples(index=False):
    with open(f'{patent_number}.txt') as patent_claim:
        patent_text.append(patent_claim.read())

In [6]:
# patent_text[0]

In [7]:
df['patent_text'] = patent_text
df.head()

,patent_number,patent_title,patent_text
0,RE30525,Extended range hydraulic transmission,Claims (1)\nHide Dependent \nI claim. 1. An ex...
1,RE30135,Electric fail-safe actuator,Claims (13)\nHide Dependent \nI claim: .[.1. A...
2,RE29872,Differential gear mechanism,Claims (13)\nHide Dependent \nWhat is claimed ...
3,RE30334,Pressure compensated hermetically sealed trans...,Claims (15)\nHide Dependent \nWhat is claimed ...
4,RE30120,Lobe type pump adjustment,Claims (2)\nHide Dependent \nI claim: .[.1. A ...


In [8]:
df.to_csv("../csv_output/mechanism_nlp_complete.csv", index=False)

In [9]:
!pip install spacy

In [10]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [11]:
import en_core_web_lg

In [12]:
TOP_N = 20
TFIDF_THRESHOLD = 0.1

def get_keywords(corpus, nlp, top_n=TOP_N, threshold=TFIDF_THRESHOLD):
    t0 = time.time()

    vec = TfidfVectorizer(ngram_range=(1, 2), stop_words=STOPWORDS)
    X = vec.fit_transform(corpus)
    print(f"X.shape: {X.shape}")

    terms = np.array(vec.get_feature_names())

    tfidfs, keywords = [], []
    all_keywords = set()
    N = len(corpus)

    for i, text in enumerate(corpus):
        D = X.getrow(i)
        D = np.squeeze(D.toarray())
        ind = np.argsort(D)[::-1]
        ind = ind[:top_n]

        D = D[ind]
        D = D[D > threshold]
        kw = terms[ind][:len(D)]

        doc = nlp(text)
        noun_phrases = [chunk.text for chunk in doc.noun_chunks if len(chunk.text.split())>1]

        D  = [d for d, w in zip(D, kw) if (w in noun_phrases or len(w.split())==1) and not has_numbers(w)]
        kw = [w for w in kw if (w in noun_phrases or len(w.split())==1) and not has_numbers(w)]

        tfidfs.append(D)
        keywords.append(kw)

        for word in kw:
            all_keywords.add(word)

        if i%1000==0 and i>0:
            print(f"({os.getpid()}) Items processed: {i :,}/{N:,}; {(time.time()-t0)/60 :.1f} minutes")

    return terms, keywords, tfidfs, all_keywords


In [13]:
import re

RE_D = re.compile('\d')

def has_numbers(string, regex_pattern=RE_D):
    return bool(regex_pattern.search(string))

In [14]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

import numpy as np

nlp = en_core_web_lg.load()
terms, keywords, tfidfs, all_keywords = get_keywords(df['patent_text'], nlp)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gongl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


X.shape: (50, 19155)


In [15]:
df['patent_keywords'] = keywords

In [16]:
df.head()

,patent_number,patent_title,patent_text,patent_keywords
0,RE30525,Extended range hydraulic transmission,Claims (1)\nHide Dependent \nI claim. 1. An ex...,"[said, means, hydraulic, shaft, output, interm..."
1,RE30135,Electric fail-safe actuator,Claims (13)\nHide Dependent \nI claim: .[.1. A...,"[electric, said, motor, valve, actuator, elect..."
2,RE29872,Differential gear mechanism,Claims (13)\nHide Dependent \nWhat is claimed ...,"[said, cam, cam member, differential gear, dif..."
3,RE30334,Pressure compensated hermetically sealed trans...,Claims (15)\nHide Dependent \nWhat is claimed ...,"[transmission means, said, environment, transm..."
4,RE30120,Lobe type pump adjustment,Claims (2)\nHide Dependent \nI claim: .[.1. A ...,"[hub, said, abutment, shaft, spring, connectio..."


In [17]:
df.to_csv("../csv_output/mechanism_nlp_complete_withkeywords.csv", index=False)

In [18]:
!nvidia-smi

Mon Aug 02 21:48:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.66       Driver Version: 452.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 3000    WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P8    11W /  N/A |    748MiB /  6144MiB |     37%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
%%time
%%capture
!pip install transformers

Wall time: 2.54 s


In [20]:
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [21]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.9.0+cu102


In [72]:
# Configurations
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 400  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [73]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [74]:
data = dict()
for patent_number, patent_title, patent_text, patent_keywords in df.itertuples(index=False):
    data[patent_number]=[patent_title, patent_text, patent_keywords] 


print(f"Number of articles: {len(data) :,}")

Number of articles: 50


In [75]:
# data [patent_title, patent_text, patent_keywords] !!!!!

In [76]:
%%time

all_keywords = set()
for v in keywords:
    for w in v:
        all_keywords.add(w)


print(f"Number of unique keywords: {len(all_keywords) :,}")  

Number of unique keywords: 440
Wall time: 0 ns


In [77]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text, keywords = [], [], []
#       title, text, keywords can be cat, dog, elephant 
        for k, v in data.items():
            title.append(v[0])
            text.append(v[1])
            keywords.append(v[2])

        self.randomize = randomize
        self.tokenizer = tokenizer 
        self.title     = title
        self.text      = text
        self.keywords  = keywords  

    #---------------------------------------------#

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#
#   get item is to get each row
    def __getitem__(self, i):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)
        
        input = SPECIAL_TOKENS['bos_token'] + self.title[i] + \
                SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token'] + \
                self.text[i] + SPECIAL_TOKENS['eos_token']

        encodings_dict = tokenizer(input,                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids), 
                'attention_mask': torch.tensor(attention_mask)}
# tensor is multi-dimension of data. converting words to numbers, then to tensors

In [78]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # Split into training and validation sets    
    train_size = int(S * len(data))

    train_ids = ids[:train_size]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

In [79]:
# Loading Tokenizer, Config and Model under transformer. Model itself like random forest!!!!!!!

def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [80]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at C:\Users\gongl/.cache\huggingface\transformers\fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true

Special tokens added


loading weights file https://huggingface.co/gpt2/resolve/main/pytorch_model.bin from cache at C:\Users\gongl/.cache\huggingface\transformers\752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


Wall time: 3.48 s


In [81]:
# - Freeze selective layers:
# - Freeze all layers except last n:
# Configurations- refer!!!!!!
# MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl} !!!!!!!!!

for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

In [82]:
train_data, val_data = split_data(data)

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

'There are 40 samples for training, and 10 samples for validation testing'

In [83]:
# Fine-tune GPT2 using Trainer

In [84]:
# ??TrainingArguments

In [85]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

In [86]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [87]:
%%time

training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=True,     
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()    

# model.fit in sklearn !!!!

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 40
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 4
C:\Users\gongl\Anaconda3\lib\site-packages\transformers\trainer.py:1308: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(


Epoch,Training Loss,Validation Loss
1,No log,72.582924
2,No log,72.582924
3,No log,72.582924
4,No log,72.582924


***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
Saving model checkpoint to /content/checkpoint-1
Configuration saved in /content/checkpoint-1\config.json
Model weights saved in /content/checkpoint-1\pytorch_model.bin
tokenizer config file saved in /content/checkpoint-1\tokenizer_config.json
Special tokens file saved in /content/checkpoint-1\special_tokens_map.json
C:\Users\gongl\Anaconda3\lib\site-packages\transformers\trainer.py:1308: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
Saving model checkpoint to /content/checkpoint-2
Configuration saved in /content/checkpoint-2\config.json
Model weights saved in /conte

Wall time: 20.7 s


In [88]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='pytorch_model.bin')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at C:\Users\gongl/.cache\huggingface\transformers\fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true

Special tokens added


loading weights file https://huggingface.co/gpt2/resolve/main/pytorch_model.bin from cache at C:\Users\gongl/.cache\huggingface\transformers\752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


FileNotFoundError: [Errno 2] No such file or directory: 'pytorch_model.bin'